# Matrix Capsule for MNIST image classification

In [2]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
from config import cfg
import numpy as np

In [3]:
iter_routing = 2
ac_lambda0 = 0.01, 
#'\lambda in the activation function a_c, iteration 0')
ac_lambda_step = 0.01,
#'It is described that \lambda increases at each iteration with a fixed schedule, however specific super parameters is absent.')
epsilon = 1e-9

batch_size = 50
################################
A = 32 # , 'number of channels in output from ReLU Conv1')
B = 8 # , 'number of capsules in output from PrimaryCaps')
C = 16 #, 'number of channels in output from ConvCaps1')
D = 16 # , 'number of channels in output from ConvCaps2')
is_train = True
num_classes = 10

In [4]:
def cross_ent_loss(output, x, y):
    loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=output)
    loss = tf.reduce_mean(loss)
    num_class = int(output.get_shape()[-1])
    data_size = int(x.get_shape()[1])

    # reconstruction loss
    y = tf.one_hot(y, num_class, dtype=tf.float32)
    y = tf.expand_dims(y, axis=2)
    output = tf.expand_dims(output, axis=2)
    output = tf.reshape(tf.multiply(output, y), shape=[cfg.batch_size, -1])
    tf.logging.info("decoder input value dimension:{}".format(output.get_shape()))

    with tf.variable_scope('decoder'):
        output = slim.fully_connected(output, 512, trainable=True)
        output = slim.fully_connected(output, 1024, trainable=True)
        output = slim.fully_connected(output, data_size * data_size,
                                      trainable=True, activation_fn=tf.sigmoid)

        x = tf.reshape(x, shape=[cfg.batch_size, -1])
        reconstruction_loss = tf.reduce_mean(tf.square(output - x))

    # regularization loss
    regularization = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
    # loss+0.0005*reconstruction_loss+regularization#
    loss_all = tf.add_n([loss] + [0.0005 * reconstruction_loss] + regularization)

    return loss_all, reconstruction_loss, output


def spread_loss(output, pose_out, x, y, m):
    """
    # check NaN
    # See: https://stackoverflow.com/questions/40701712/how-to-check-nan-in-gradients-in-tensorflow-when-updating
    output_check = [tf.check_numerics(output, message='NaN Found!')]
    with tf.control_dependencies(output_check):
    """

    num_class = int(output.get_shape()[-1])
    data_size = int(x.get_shape()[1])

    y = tf.one_hot(y, num_class, dtype=tf.float32)

    # spread loss
    output1 = tf.reshape(output, shape=[cfg.batch_size, 1, num_class])
    y = tf.expand_dims(y, axis=2)
    at = tf.matmul(output1, y)
    """Paper eq(5)."""
    loss = tf.square(tf.maximum(0., m - (at - output1)))
    loss = tf.matmul(loss, 1. - y)
    loss = tf.reduce_mean(loss)

    # reconstruction loss
    # pose_out = tf.reshape(tf.matmul(pose_out, y, transpose_a=True), shape=[cfg.batch_size, -1])
    pose_out = tf.reshape(tf.multiply(pose_out, y), shape=[cfg.batch_size, -1])
    tf.logging.info("decoder input value dimension:{}".format(pose_out.get_shape()))

    with tf.variable_scope('decoder'):
        pose_out = slim.fully_connected(pose_out, 512, trainable=True, weights_regularizer=tf.contrib.layers.l2_regularizer(5e-04))
        pose_out = slim.fully_connected(pose_out, 1024, trainable=True, weights_regularizer=tf.contrib.layers.l2_regularizer(5e-04))
        pose_out = slim.fully_connected(pose_out, data_size * data_size,
                                        trainable=True, activation_fn=tf.sigmoid, weights_regularizer=tf.contrib.layers.l2_regularizer(5e-04))

        x = tf.reshape(x, shape=[cfg.batch_size, -1])
        reconstruction_loss = tf.reduce_mean(tf.square(pose_out - x))

    if cfg.weight_reg:
        # regularization loss
        regularization = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
        # loss+0.0005*reconstruction_loss+regularization#
        loss_all = tf.add_n([loss] + [0.0005 * data_size* data_size * reconstruction_loss] + regularization)
    else:
        loss_all = tf.add_n([loss] + [0.0005 * data_size* data_size * reconstruction_loss])

    return loss_all, loss, reconstruction_loss, pose_out

# input should be a tensor with size as [batch_size, height, width, channels]


def kernel_tile(input, kernel, stride):
    # output = tf.extract_image_patches(input, ksizes=[1, kernel, kernel, 1], strides=[1, stride, stride, 1], rates=[1, 1, 1, 1], padding='VALID')

    input_shape = input.get_shape()
    tile_filter = np.zeros(shape=[kernel, kernel, input_shape[3],
                                  kernel * kernel], dtype=np.float32)
    for i in range(kernel):
        for j in range(kernel):
            tile_filter[i, j, :, i * kernel + j] = 1.0

    tile_filter_op = tf.constant(tile_filter, dtype=tf.float32)
    output = tf.nn.depthwise_conv2d(input, tile_filter_op, strides=[
                                    1, stride, stride, 1], padding='VALID')
    output_shape = output.get_shape()
    output = tf.reshape(output, shape=[int(output_shape[0]), int(
        output_shape[1]), int(output_shape[2]), int(input_shape[3]), kernel * kernel])
    output = tf.transpose(output, perm=[0, 1, 2, 4, 3])

    return output

# input should be a tensor with size as [batch_size, caps_num_i, 16]
def mat_transform(input, caps_num_c, regularizer, tag=False):
    batch_size = int(input.get_shape()[0])
    caps_num_i = int(input.get_shape()[1])
    output = tf.reshape(input, shape=[batch_size, caps_num_i, 1, 4, 4])
    # the output of capsule is miu, the mean of a Gaussian, and activation, the sum of probabilities
    # it has no relationship with the absolute values of w and votes
    # using weights with bigger stddev helps numerical stability
    w = slim.variable('w', shape=[1, caps_num_i, caps_num_c, 4, 4], dtype=tf.float32,
                      initializer=tf.truncated_normal_initializer(mean=0.0, stddev=1.0),
                      regularizer=regularizer)

    w = tf.tile(w, [batch_size, 1, 1, 1, 1])
    output = tf.tile(output, [1, 1, caps_num_c, 1, 1])
    votes = tf.reshape(tf.matmul(output, w), [batch_size, caps_num_i, caps_num_c, 16])

    return votes


def build_arch_baseline(input, is_train: bool, num_classes: int):

    bias_initializer = tf.truncated_normal_initializer(
        mean=0.0, stddev=0.01)  # tf.constant_initializer(0.0)
    # The paper didnot mention any regularization, a common l2 regularizer to weights is added here
    weights_regularizer = tf.contrib.layers.l2_regularizer(5e-04)

    tf.logging.info('input shape: {}'.format(input.get_shape()))

    # weights_initializer=initializer,
    with slim.arg_scope([slim.conv2d, slim.fully_connected], trainable=is_train, biases_initializer=bias_initializer, weights_regularizer=weights_regularizer):
        with tf.variable_scope('relu_conv1') as scope:
            output = slim.conv2d(input, num_outputs=32, kernel_size=[
                                 5, 5], stride=1, padding='SAME', scope=scope, activation_fn=tf.nn.relu)
            output = slim.max_pool2d(output, [2, 2], scope='max_2d_layer1')

            tf.logging.info('output shape: {}'.format(output.get_shape()))

        with tf.variable_scope('relu_conv2') as scope:
            output = slim.conv2d(output, num_outputs=64, kernel_size=[
                                 5, 5], stride=1, padding='SAME', scope=scope, activation_fn=tf.nn.relu)
            output = slim.max_pool2d(output, [2, 2], scope='max_2d_layer2')

            tf.logging.info('output shape: {}'.format(output.get_shape()))

        output = slim.flatten(output)
        output = slim.fully_connected(output, 1024, scope='relu_fc3', activation_fn=tf.nn.relu)
        tf.logging.info('output shape: {}'.format(output.get_shape()))
        output = slim.dropout(output, 0.5, scope='dp')
        output = slim.fully_connected(output, num_classes, scope='final_layer', activation_fn=None)
        tf.logging.info('output shape: {}'.format(output.get_shape()))
        return output

In [5]:
def em_routing(votes, activation, caps_num_c, regularizer, tag=False):
    test = []

    batch_size = int(votes.get_shape()[0])
    caps_num_i = int(activation.get_shape()[1])
    n_channels = int(votes.get_shape()[-1])

    sigma_square = []
    miu = []
    activation_out = []
    beta_v = slim.variable('beta_v', shape=[caps_num_c, n_channels], dtype=tf.float32,
                           initializer=tf.constant_initializer(0.0),#tf.truncated_normal_initializer(mean=0.0, stddev=0.01),
                           regularizer=regularizer)
    beta_a = slim.variable('beta_a', shape=[caps_num_c], dtype=tf.float32,
                           initializer=tf.constant_initializer(0.0),#tf.truncated_normal_initializer(mean=0.0, stddev=0.01),
                           regularizer=regularizer)

    # votes_in = tf.stop_gradient(votes, name='stop_gradient_votes')
    # activation_in = tf.stop_gradient(activation, name='stop_gradient_activation')
    votes_in = votes
    activation_in = activation

    for iters in range(iter_routing):
        # if iters == cfg.iter_routing-1:

        # e-step
        if iters == 0:
            r = tf.constant(np.ones([batch_size, caps_num_i, caps_num_c], dtype=np.float32) / caps_num_c)
        else:
            # Contributor: Yunzhi Shi
            # log and exp here provide higher numerical stability especially for bigger number of iterations
            log_p_c_h = -tf.log(tf.sqrt(sigma_square)) - \
                        (tf.square(votes_in - miu) / (2 * sigma_square))
            log_p_c_h = log_p_c_h - \
                        (tf.reduce_max(log_p_c_h, axis=[2, 3], keep_dims=True) - tf.log(10.0))
            p_c = tf.exp(tf.reduce_sum(log_p_c_h, axis=3))

            ap = p_c * tf.reshape(activation_out, shape=[batch_size, 1, caps_num_c])

            # ap = tf.reshape(activation_out, shape=[batch_size, 1, caps_num_c])

            r = ap / (tf.reduce_sum(ap, axis=2, keepdims=True) + epsilon)

        # m-step
        r = r * activation_in
        r = r / (tf.reduce_sum(r, axis=2, keepdims=True)+epsilon)

        r_sum = tf.reduce_sum(r, axis=1, keepdims=True)
        r1 = tf.reshape(r / (r_sum + epsilon),
                        shape=[batch_size, caps_num_i, caps_num_c, 1])

        miu = tf.reduce_sum(votes_in * r1, axis=1, keepdims=True)
        sigma_square = tf.reduce_sum(tf.square(votes_in - miu) * r1,
                                     axis=1, keepdims=True) + epsilon

        if iters == iter_routing-1:
            r_sum = tf.reshape(r_sum, [batch_size, caps_num_c, 1])
            cost_h = (beta_v + tf.log(tf.sqrt(tf.reshape(sigma_square,
                                                         shape=[batch_size, caps_num_c, n_channels])))) * r_sum

            activation_out = tf.nn.softmax(ac_lambda0 * (beta_a - tf.reduce_sum(cost_h, axis=2)))
        else:
            activation_out = tf.nn.softmax(r_sum)
        # if iters <= cfg.iter_routing-1:
        #     activation_out = tf.stop_gradient(activation_out, name='stop_gradient_activation')

    return miu, activation_out, test


In [6]:
def get_coord_add(dataset_name: str):
    import numpy as np
    # TODO: get coord add for cifar10/100 datasets (32x32x3)
    options = {'mnist': ([[[8., 8.], [12., 8.], [16., 8.]],
                          [[8., 12.], [12., 12.], [16., 12.]],
                          [[8., 16.], [12., 16.], [16., 16.]]], 28.),
               'smallNORB': ([[[8., 8.], [12., 8.], [16., 8.], [24., 8.]],
                              [[8., 12.], [12., 12.], [16., 12.], [24., 12.]],
                              [[8., 16.], [12., 16.], [16., 16.], [24., 16.]],
                              [[8., 24.], [12., 24.], [16., 24.], [24., 24.]]], 32.)
               }
    coord_add, scale = options[dataset_name]

    coord_add = np.array(coord_add, dtype=np.float32) / scale

    return coord_add

In [7]:
#coord_add = get_coord_add('mnist') 

In [8]:
iter_routing = 2
ac_lambda0 = 0.01, 
#'\lambda in the activation function a_c, iteration 0')
ac_lambda_step = 0.01,
#'It is described that \lambda increases at each iteration with a fixed schedule, however specific super parameters is absent.')
epsilon = 1e-9

batch_size = 50
################################
A = 32 # , 'number of channels in output from ReLU Conv1')
B = 8 # , 'number of capsules in output from PrimaryCaps')
C = 16 #, 'number of channels in output from ConvCaps1')
D = 16 # , 'number of channels in output from ConvCaps2')
is_train = True
num_classes = 10

In [9]:
class MatrixCapsule(object):
    def __init__(self, is_train=True):
        tf.reset_default_graph()
        self.graph = tf.Graph()
        self._build_arch()
        self.sess = tf.Session()
        self.sess.run(tf.global_variables_initializer())
        print("Graph for Matrix Capsule is ready for training")
    def _build_arch(self):
        self.X = tf.placeholder(tf.float32, shape=(batch_size, 28, 28, 1))
        self.Y = tf.placeholder(tf.float32, [batch_size, 10])
        data_size = int(self.X.get_shape()[1])
        # xavier initialization is necessary here to provide higher stability
        initializer = tf.truncated_normal_initializer(mean=0.0, stddev=0.01)
        # instead of initializing bias with constant 0, 
        # a truncated normal initializer is exploited here for higher stability 
        bias_initializer = tf.truncated_normal_initializer(mean=0.0, stddev=0.01)  # tf.constant_initializer(0.0)
        # The paper didnot mention any regularization, a common l2 regularizer to weights is added here
        weights_regularizer = tf.contrib.layers.l2_regularizer(5e-04)
        # weights_initializer=initializer,
        with slim.arg_scope([slim.conv2d], trainable=is_train, biases_initializer=bias_initializer, weights_regularizer=weights_regularizer):
            with tf.variable_scope('relu_conv1'):
                output = slim.conv2d(self.X, num_outputs=A, kernel_size=[5, 5], stride=2, padding='VALID', scope='relu_conv1', activation_fn=tf.nn.relu)
                data_size = int(np.floor((data_size - 4) / 2))
                #print(output.get_shape())
                #print(data_size)
                a = [batch_size, data_size, data_size, 32]
                #print(a)
                assert output.get_shape() == [batch_size, data_size, data_size, 32]
            with tf.variable_scope('primary_caps'):
                pose = slim.conv2d(output, num_outputs=B * 16,kernel_size=[1, 1], stride=1, padding='VALID', scope='primary_caps', activation_fn=None)
                activation = slim.conv2d(output, num_outputs=B, kernel_size=[
                                         1, 1], stride=1, padding='VALID', scope='primary_caps/activation', activation_fn=tf.nn.sigmoid)
                pose = tf.reshape(pose, shape=[batch_size, data_size, data_size, B, 16])
                activation = tf.reshape(activation, shape=[batch_size, data_size, data_size, B, 1])
                output = tf.concat([pose, activation], axis=4)
                output = tf.reshape(output, shape=[batch_size, data_size, data_size, -1])
                assert output.get_shape() == [batch_size, data_size, data_size, B * 17]

            with tf.variable_scope('conv_caps1') as scope:
                output = kernel_tile(output, 3, 2)
                data_size = int(np.floor((data_size - 2) / 2))
                output = tf.reshape(output, shape=[batch_size *
                                                   data_size * data_size, 3 * 3 * B, 17])
                activation = tf.reshape(output[:, :, 16], shape=[
                                        batch_size * data_size * data_size, 3 * 3 * B, 1])
                with tf.variable_scope('v') as scope:
                    votes = mat_transform(output[:, :, :16], C, weights_regularizer, tag=True)
                with tf.variable_scope('routing') as scope:
                    miu, activation, _ = em_routing(votes, activation, C, weights_regularizer)
                pose = tf.reshape(miu, shape=[batch_size, data_size, data_size, C, 16])
                activation = tf.reshape(activation, shape=[batch_size, data_size, data_size, C, 1])
                output = tf.reshape(tf.concat([pose, activation], axis=4),[batch_size, data_size, data_size, -1])
                #print(output.get_shape())
            with tf.variable_scope('conv_caps2') as scope:
                output = kernel_tile(output, 3, 1)
                data_size = int(np.floor((data_size - 2) / 1))
                output = tf.reshape(output, shape=[batch_size * data_size * data_size, 3 * 3 * C, 17])
                activation = tf.reshape(output[:, :, 16], shape=[batch_size * data_size * data_size, 3 * 3 * C, 1])
                with tf.variable_scope('v') as scope:
                    votes = mat_transform(output[:, :, :16], D, weights_regularizer)
                    #tf.logging.info('conv cap 2 votes shape: {}'.format(votes.get_shape()))
                with tf.variable_scope('routing') as scope:
                    miu, activation, _ = em_routing(votes, activation, D, weights_regularizer)
                pose = tf.reshape(miu, shape=[batch_size * data_size * data_size, D, 16])
                #tf.logging.info('conv cap 2 pose shape: {}'.format(votes.get_shape()))
                activation = tf.reshape(activation, shape=[batch_size * data_size * data_size, D, 1])
                #tf.logging.info('conv cap 2 activation shape: {}'.format(activation.get_shape()))
                # It is not clear from the paper that ConvCaps2 is full connected to Class Capsules,
                #or is conv connected with kernel size of 1*1 and a global average pooling.
                # From the description in Figure 1 of the paper and the amount of parameters 
                #(310k in the paper and 316,853 in fact), I assume a conv cap plus a golbal average pooling is the design.
            with tf.variable_scope('class_caps') as scope:
                with tf.variable_scope('v') as scope:
                    votes = mat_transform(pose, num_classes, weights_regularizer)

                    assert votes.get_shape() == [batch_size * data_size * data_size, D, num_classes, 16]
                    #tf.logging.info('class cap votes original shape: {}'.format(votes.get_shape()))
                    coord_add = get_coord_add('mnist') 
                    coord_add = np.reshape(coord_add, newshape=[data_size * data_size, 1, 1, 2])
                    coord_add = np.tile(coord_add, [batch_size, D, num_classes, 1])
                    coord_add_op = tf.constant(coord_add, dtype=tf.float32)

                    votes = tf.concat([coord_add_op, votes], axis=3)
                    #tf.logging.info('class cap votes coord add shape: {}'.format(votes.get_shape()))

                with tf.variable_scope('routing') as scope:
                    miu, activation, test2 = em_routing(votes, activation, num_classes, weights_regularizer)
                    #tf.logging.info('class cap activation shape: {}'.format(activation.get_shape()))
                    #tf.summary.histogram(name="class_cap_routing_hist", values=test2)

                output = tf.reshape(activation, shape=[batch_size, data_size, data_size, num_classes])
            output = tf.reshape(tf.nn.avg_pool(output, ksize=[1, data_size, data_size, 1], strides=[
                        1, 1, 1, 1], padding='VALID'), shape=[batch_size, num_classes])
            #tf.logging.info('class cap output shape: {}'.format(output.get_shape()))

            pose = tf.nn.avg_pool(tf.reshape(miu, shape=[batch_size, data_size, data_size, -1]), ksize=[
                                  1, data_size, data_size, 1], strides=[1, 1, 1, 1], padding='VALID')
            pose_out = tf.reshape(pose, shape=[batch_size, num_classes, 18])
            vector_j = tf.reshape(pose_out, shape= [-1, num_classes * 18])
            #print("output_size",pose_out.get_shape())
            #print("output_size",vector_j.get_shape())
            
            with tf.variable_scope('output_layer') as scope:
                self.logits = tf.contrib.layers.fully_connected(vector_j, num_outputs=10, activation_fn=None)
                
        with tf.variable_scope('loss') as scope:
            self.cross_entropy_loss = tf.losses.softmax_cross_entropy(self.Y, self.logits)
            #print("output_size",self.logits.get_shape())
            
        with tf.variable_scope('optimizer') as scope:
            self.train_op = tf.train.AdamOptimizer(1e-3).minimize(self.cross_entropy_loss)
            
        with tf.variable_scope('prediction') as scope:  
            prediction = tf.nn.softmax(self.logits,-1)
            self.y_hat = tf.to_int32(tf.argmax(prediction, axis=1))
        
        with tf.variable_scope('acc'):
            self.labels = tf.to_int32(tf.argmax(self.Y, axis=1))
            correct_prediction = tf.equal(tf.to_int32(self.labels), self.y_hat)
            self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32)) * 100
        #print(self.accuracy) 
    def train(self, batch_xs, batch_ys):
        _,loss,acc = self.sess.run([self.train_op,self.cross_entropy_loss, self.accuracy],feed_dict={self.X: batch_xs, self.Y: batch_ys})
        return loss, acc

In [13]:
n_itrs=50*55000/50
n_itrs

55000.0

In [10]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
n_itrs = 7
model = MatrixCapsule()
print("session started")

n_itrs = 1000
for step in range(n_itrs):
    batch_xs, batch_ys = mnist.train.next_batch(batch_size)
    batch_xs = batch_xs.reshape(batch_size, 28, 28,1)
    batch_ys = batch_ys.reshape(batch_size, 10)
    loss, acc =  model.train(batch_xs,batch_ys)
    assert not np.isnan(loss), 'Something wrong! loss is nan...'
    print(step)
    if (step % 30 == 0):
        print("step-{} summary: loss= {}, training accuracy = {}".format(step, loss,acc))

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Graph for Matrix Capsule is ready for training
session started
0
step-0 summary: loss= 2.3272855281829834, training accuracy = 6.0
1
2
3
4
5
step-5 summary: loss= 2.3497073650360107, training accuracy = 6.0
6


In [50]:
MC = MatrixCapsule()

Tensor("acc/mul:0", shape=(), dtype=float32)
Graph for Matrix Capsule is ready for training


In [55]:
'''batch_size = 50
################################
A = 32 # , 'number of channels in output from ReLU Conv1')
B = 8 # , 'number of capsules in output from PrimaryCaps')
C = 16 #, 'number of channels in output from ConvCaps1')
D = 16 # , 'number of channels in output from ConvCaps2')
tf.reset_default_graph()
input = tf.placeholder(tf.float32, shape=(50, 28, 28, 1))
y = tf.placeholder(tf.float32, [None, 10])
is_train = True
num_classes = 10
test1 = []
data_size = int(input.get_shape()[1])
print(data_size)
# xavier initialization is necessary here to provide higher stability
# initializer = tf.truncated_normal_initializer(mean=0.0, stddev=0.01)
# instead of initializing bias with constant 0, a truncated normal initializer is exploited here for higher stability
bias_initializer = tf.truncated_normal_initializer(mean=0.0, stddev=0.01)  # tf.constant_initializer(0.0)
# The paper didnot mention any regularization, a common l2 regularizer to weights is added here
weights_regularizer = tf.contrib.layers.l2_regularizer(5e-04)
# weights_initializer=initializer,
with slim.arg_scope([slim.conv2d], trainable=is_train, biases_initializer=bias_initializer, weights_regularizer=weights_regularizer):
    with tf.variable_scope('relu_conv1'):
        output = slim.conv2d(input, num_outputs=A, kernel_size=[5, 5], stride=2, padding='VALID', scope='relu_conv1', activation_fn=tf.nn.relu)
        data_size = int(np.floor((data_size - 4) / 2))
        print(output.get_shape())
        print(data_size)
        a = [batch_size, data_size, data_size, 32]
        print(a)
        assert output.get_shape() == [batch_size, data_size, data_size, 32]
    with tf.variable_scope('primary_caps'):
        pose = slim.conv2d(output, num_outputs=B * 16,kernel_size=[1, 1], stride=1, padding='VALID', scope='primary_caps', activation_fn=None)
        activation = slim.conv2d(output, num_outputs=B, kernel_size=[
                                 1, 1], stride=1, padding='VALID', scope='primary_caps/activation', activation_fn=tf.nn.sigmoid)
        pose = tf.reshape(pose, shape=[batch_size, data_size, data_size, B, 16])
        activation = tf.reshape(activation, shape=[batch_size, data_size, data_size, B, 1])
        output = tf.concat([pose, activation], axis=4)
        output = tf.reshape(output, shape=[batch_size, data_size, data_size, -1])
        assert output.get_shape() == [batch_size, data_size, data_size, B * 17]
    
    with tf.variable_scope('conv_caps1') as scope:
        output = kernel_tile(output, 3, 2)
        data_size = int(np.floor((data_size - 2) / 2))
        output = tf.reshape(output, shape=[batch_size *
                                           data_size * data_size, 3 * 3 * B, 17])
        activation = tf.reshape(output[:, :, 16], shape=[
                                batch_size * data_size * data_size, 3 * 3 * B, 1])
        with tf.variable_scope('v') as scope:
            votes = mat_transform(output[:, :, :16], C, weights_regularizer, tag=True)
        with tf.variable_scope('routing') as scope:
            miu, activation, _ = em_routing(votes, activation, C, weights_regularizer)
        pose = tf.reshape(miu, shape=[batch_size, data_size, data_size, C, 16])
        activation = tf.reshape(activation, shape=[batch_size, data_size, data_size, C, 1])
        output = tf.reshape(tf.concat([pose, activation], axis=4),[batch_size, data_size, data_size, -1])
        print(output.get_shape())
    with tf.variable_scope('conv_caps2') as scope:
        output = kernel_tile(output, 3, 1)
        data_size = int(np.floor((data_size - 2) / 1))
        output = tf.reshape(output, shape=[batch_size * data_size * data_size, 3 * 3 * C, 17])
        activation = tf.reshape(output[:, :, 16], shape=[batch_size * data_size * data_size, 3 * 3 * C, 1])
        with tf.variable_scope('v') as scope:
            votes = mat_transform(output[:, :, :16], D, weights_regularizer)
            #tf.logging.info('conv cap 2 votes shape: {}'.format(votes.get_shape()))
        with tf.variable_scope('routing') as scope:
            miu, activation, _ = em_routing(votes, activation, D, weights_regularizer)
        pose = tf.reshape(miu, shape=[batch_size * data_size * data_size, D, 16])
        #tf.logging.info('conv cap 2 pose shape: {}'.format(votes.get_shape()))
        activation = tf.reshape(activation, shape=[batch_size * data_size * data_size, D, 1])
        #tf.logging.info('conv cap 2 activation shape: {}'.format(activation.get_shape()))
        # It is not clear from the paper that ConvCaps2 is full connected to Class Capsules,
        #or is conv connected with kernel size of 1*1 and a global average pooling.
        # From the description in Figure 1 of the paper and the amount of parameters 
        #(310k in the paper and 316,853 in fact), I assume a conv cap plus a golbal average pooling is the design.
    with tf.variable_scope('class_caps') as scope:
        with tf.variable_scope('v') as scope:
            votes = mat_transform(pose, num_classes, weights_regularizer)

            assert votes.get_shape() == [batch_size * data_size * data_size, D, num_classes, 16]
            #tf.logging.info('class cap votes original shape: {}'.format(votes.get_shape()))
            coord_add = get_coord_add('mnist') 
            coord_add = np.reshape(coord_add, newshape=[data_size * data_size, 1, 1, 2])
            coord_add = np.tile(coord_add, [batch_size, D, num_classes, 1])
            coord_add_op = tf.constant(coord_add, dtype=tf.float32)

            votes = tf.concat([coord_add_op, votes], axis=3)
            #tf.logging.info('class cap votes coord add shape: {}'.format(votes.get_shape()))

        with tf.variable_scope('routing') as scope:
            miu, activation, test2 = em_routing(votes, activation, num_classes, weights_regularizer)
            #tf.logging.info('class cap activation shape: {}'.format(activation.get_shape()))
            #tf.summary.histogram(name="class_cap_routing_hist", values=test2)

        output = tf.reshape(activation, shape=[batch_size, data_size, data_size, num_classes])

    output = tf.reshape(tf.nn.avg_pool(output, ksize=[1, data_size, data_size, 1], strides=[
                        1, 1, 1, 1], padding='VALID'), shape=[batch_size, num_classes])
    #tf.logging.info('class cap output shape: {}'.format(output.get_shape()))

    pose = tf.nn.avg_pool(tf.reshape(miu, shape=[batch_size, data_size, data_size, -1]), ksize=[
                          1, data_size, data_size, 1], strides=[1, 1, 1, 1], padding='VALID')
    pose_out = tf.reshape(pose, shape=[batch_size, num_classes, 18])
    vector_j = tf.reshape(pose_out, shape= [-1, num_classes * 18])
    print("output_size",pose_out.get_shape())
    print("output_size",vector_j.get_shape())
logits = tf.contrib.layers.fully_connected(vector_j, num_outputs=10, activation_fn=None)

cross_entropy_loss = tf.losses.softmax_cross_entropy(y,logits)
    
print("output_size",logits.get_shape())
prediction = tf.nn.softmax(logits,-1)
y_hat = tf.to_int32(tf.argmax(prediction, axis=1))
train_op = tf.train.AdamOptimizer(1e-3).minimize(cross_entropy_loss)
# sess.run(tf.global_variables_initializer())
print(y_hat)
print(y)'''
a=10

In [56]:
total_parameters = 0
for variable in tf.trainable_variables():
    # shape is an array of tf.Dimension
    shape = variable.get_shape()
    #print(shape)
    #print(len(shape))
    variable_parameters = 1
    for dim in shape:
        #print(dim)
        variable_parameters *= dim.value
    #print(variable_parameters)
    total_parameters += variable_parameters
print(total_parameters)

65720


In [57]:
'''
batch_size = 50
################################
A = 32 # , 'number of channels in output from ReLU Conv1')
B = 8 # , 'number of capsules in output from PrimaryCaps')
C = 16 #, 'number of channels in output from ConvCaps1')
D = 16 # , 'number of channels in output from ConvCaps2')
tf.reset_default_graph()
input = tf.placeholder(tf.float32, shape=(50, 28, 28, 1))
y = tf.placeholder(tf.float32, [None, 10])
is_train = True
num_classes = 10
test1 = []
data_size = int(input.get_shape()[1])
print(data_size)
# xavier initialization is necessary here to provide higher stability
# initializer = tf.truncated_normal_initializer(mean=0.0, stddev=0.01)
# instead of initializing bias with constant 0, a truncated normal initializer is exploited here for higher stability
bias_initializer = tf.truncated_normal_initializer(mean=0.0, stddev=0.01)  # tf.constant_initializer(0.0)
# The paper didnot mention any regularization, a common l2 regularizer to weights is added here
weights_regularizer = tf.contrib.layers.l2_regularizer(5e-04)
# weights_initializer=initializer,
with slim.arg_scope([slim.conv2d], trainable=is_train, biases_initializer=bias_initializer, weights_regularizer=weights_regularizer):
    with tf.variable_scope('relu_conv1'):
        output = slim.conv2d(input, num_outputs=A, kernel_size=[5, 5], stride=2, padding='VALID', scope='relu_conv1', activation_fn=tf.nn.relu)
        data_size = int(np.floor((data_size - 4) / 2))
        print(output.get_shape())
        print(data_size)
        a = [batch_size, data_size, data_size, 32]
        print(a)
        assert output.get_shape() == [batch_size, data_size, data_size, 32]
    with tf.variable_scope('primary_caps'):
        pose = slim.conv2d(output, num_outputs=B * 16,kernel_size=[1, 1], stride=1, padding='VALID', scope='primary_caps', activation_fn=None)
        activation = slim.conv2d(output, num_outputs=B, kernel_size=[
                                 1, 1], stride=1, padding='VALID', scope='primary_caps/activation', activation_fn=tf.nn.sigmoid)
        pose = tf.reshape(pose, shape=[batch_size, data_size, data_size, B, 16])
        activation = tf.reshape(activation, shape=[batch_size, data_size, data_size, B, 1])
        output = tf.concat([pose, activation], axis=4)
        output = tf.reshape(output, shape=[batch_size, data_size, data_size, -1])
        assert output.get_shape() == [batch_size, data_size, data_size, B * 17]
    
    with tf.variable_scope('conv_caps1') as scope:
        output = kernel_tile(output, 3, 2)
        data_size = int(np.floor((data_size - 2) / 2))
        output = tf.reshape(output, shape=[batch_size *
                                           data_size * data_size, 3 * 3 * B, 17])
        activation = tf.reshape(output[:, :, 16], shape=[
                                batch_size * data_size * data_size, 3 * 3 * B, 1])
        with tf.variable_scope('v') as scope:
            votes = mat_transform(output[:, :, :16], C, weights_regularizer, tag=True)
        with tf.variable_scope('routing') as scope:
            miu, activation, _ = em_routing(votes, activation, C, weights_regularizer)
        pose = tf.reshape(miu, shape=[batch_size, data_size, data_size, C, 16])
        activation = tf.reshape(activation, shape=[batch_size, data_size, data_size, C, 1])
        output = tf.reshape(tf.concat([pose, activation], axis=4),[batch_size, data_size, data_size, -1])
        print(output.get_shape())
    with tf.variable_scope('conv_caps2') as scope:
        output = kernel_tile(output, 3, 1)
        data_size = int(np.floor((data_size - 2) / 1))
        output = tf.reshape(output, shape=[batch_size * data_size * data_size, 3 * 3 * C, 17])
        activation = tf.reshape(output[:, :, 16], shape=[batch_size * data_size * data_size, 3 * 3 * C, 1])
        with tf.variable_scope('v') as scope:
            votes = mat_transform(output[:, :, :16], D, weights_regularizer)
            #tf.logging.info('conv cap 2 votes shape: {}'.format(votes.get_shape()))
        with tf.variable_scope('routing') as scope:
            miu, activation, _ = em_routing(votes, activation, D, weights_regularizer)
        pose = tf.reshape(miu, shape=[batch_size * data_size * data_size, D, 16])
        #tf.logging.info('conv cap 2 pose shape: {}'.format(votes.get_shape()))
        activation = tf.reshape(activation, shape=[batch_size * data_size * data_size, D, 1])
        #tf.logging.info('conv cap 2 activation shape: {}'.format(activation.get_shape()))
        # It is not clear from the paper that ConvCaps2 is full connected to Class Capsules,
        #or is conv connected with kernel size of 1*1 and a global average pooling.
        # From the description in Figure 1 of the paper and the amount of parameters 
        #(310k in the paper and 316,853 in fact), I assume a conv cap plus a golbal average pooling is the design.
    with tf.variable_scope('class_caps') as scope:
        with tf.variable_scope('v') as scope:
            votes = mat_transform(pose, num_classes, weights_regularizer)

            assert votes.get_shape() == [batch_size * data_size * data_size, D, num_classes, 16]
            #tf.logging.info('class cap votes original shape: {}'.format(votes.get_shape()))
            coord_add = get_coord_add('mnist') 
            coord_add = np.reshape(coord_add, newshape=[data_size * data_size, 1, 1, 2])
            coord_add = np.tile(coord_add, [batch_size, D, num_classes, 1])
            coord_add_op = tf.constant(coord_add, dtype=tf.float32)

            votes = tf.concat([coord_add_op, votes], axis=3)
            #tf.logging.info('class cap votes coord add shape: {}'.format(votes.get_shape()))

        with tf.variable_scope('routing') as scope:
            miu, activation, test2 = em_routing(votes, activation, num_classes, weights_regularizer)
            #tf.logging.info('class cap activation shape: {}'.format(activation.get_shape()))
            #tf.summary.histogram(name="class_cap_routing_hist", values=test2)

        output = tf.reshape(activation, shape=[batch_size, data_size, data_size, num_classes])

    output = tf.reshape(tf.nn.avg_pool(output, ksize=[1, data_size, data_size, 1], strides=[
                        1, 1, 1, 1], padding='VALID'), shape=[batch_size, num_classes])
    #tf.logging.info('class cap output shape: {}'.format(output.get_shape()))

    pose = tf.nn.avg_pool(tf.reshape(miu, shape=[batch_size, data_size, data_size, -1]), ksize=[
                          1, data_size, data_size, 1], strides=[1, 1, 1, 1], padding='VALID')
    pose_out = tf.reshape(pose, shape=[batch_size, num_classes, 18])
    vector_j = tf.reshape(pose_out, shape= [-1, num_classes * 18])
    print("output_size",pose_out.get_shape())
    print("output_size",vector_j.get_shape())
logits = tf.contrib.layers.fully_connected(vector_j, num_outputs=10, activation_fn=None)

cross_entropy_loss = tf.losses.softmax_cross_entropy(y,logits)
    
print("output_size",logits.get_shape())
prediction = tf.nn.softmax(logits,-1)
y_hat = tf.to_int32(tf.argmax(prediction, axis=1))
train_op = tf.train.AdamOptimizer(1e-3).minimize(cross_entropy_loss)
# sess.run(tf.global_variables_initializer())
print(y_hat)
print(y)
'''
original = True